In [ ]:
pip install --upgrade transformers huggingface_hub --q

In [2]:
import os 
import json 
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification
from peft import LoraConfig, TaskType, get_peft_model

In [3]:
import torch

In [4]:
model_name="EleutherAI/pythia-410m"

In [5]:
base_model = AutoModelForCausalLM.from_pretrained(model_name,attn_implementation="sdpa",dtype=torch.float16, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/911M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [6]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.padding_side = 'left'

In [7]:
from peft import PeftModel

In [8]:
policy_model = PeftModel.from_pretrained(
    base_model,
    "/kaggle/input/loraadapters/pytorch/default/1",
    trainable=True
)

In [9]:
# config = LoraConfig(
#     r=8,
#     lora_alpha=16,
#     lora_dropout=0.1,
#     bias='none',
#     target_modules=["query_key_value"],
#     task_type=TaskType.CAUSAL_LM
# )
# lora_model = get_peft_model(model, config)
# optimizer = torch.optim.AdamW(lora_model.parameters(), lr=1e-4)

# lora_model, optimizer = accelerator.prepare(lora_model, optimizer)

# accelerator.load_state("/kaggle/input/bestmodel/pytorch/default/1")

In [10]:
# lora_model.save_pretrained("/kaggle/working/lora_adapters")

In [11]:
value_model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=1
)

Some weights of GPTNeoXForSequenceClassification were not initialized from the model checkpoint at EleutherAI/pythia-410m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
value_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias='none',
    task_type=TaskType.SEQ_CLS
)

In [13]:
value_model = get_peft_model(value_model, value_config)
value_model.to('cuda')
value_model.config.pad_token_id = tokenizer.pad_token_id

In [14]:
value_model.print_trainable_parameters()

trainable params: 1,573,888 || all params: 355,397,632 || trainable%: 0.4429


In [15]:
value_model.device


device(type='cuda', index=0)

In [16]:
ref_model = PeftModel.from_pretrained(
    base_model,
    "/kaggle/input/loraadapters/pytorch/default/1",
    trainable=False
)

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [17]:
with open("/kaggle/input/mathdataset/math_dataset.json", 'r') as f:
    dataset = json.load(f)

In [18]:
from datasets import Dataset

In [19]:
train_data = []
for item in dataset['train']:
    train_data.append({
        'question': item['question'],
        'answer': item['answer']
    })

train_dataset = Dataset.from_list(train_data)

In [20]:
def tokenize_function(sample):
    tokenized = tokenizer(
        sample['question'], 
        truncation=True, 
        padding='max_length',
        max_length=256
    )
    return {
        'input_ids': tokenized['input_ids'],
        'attention_mask': tokenized['attention_mask'],
        'answer': sample['answer'],  # Keep this!
        'question': sample['question']  # Keep this too!
    }

tokenized_dataset = train_dataset.map(
    tokenize_function,
    remove_columns=['question', 'answer']  # Remove only after copying
)

Map:   0%|          | 0/738 [00:00<?, ? examples/s]

In [21]:
tokenized_dataset

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 738
})

In [22]:
from typing import Callable, Optional, Tuple, List
from transformers import PreTrainedTokenizerBase
from tqdm import tqdm
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [23]:
def compute_reward(response: torch.Tensor, answers: List)->torch.Tensor:
    rewards = []
    for response, answer in zip(response, answers):
        response_ = response.lower().strip()
        answer_ = answer.lower().strip()

        if answer_ in response_:
            reward = 1.0
        else:
            reward= -1.0
        rewards.append(reward)
    return torch.tensor(rewards, dtype=torch.float16)

In [24]:
def compute_gae(reward: torch.Tensor, values:torch.Tensor, gamma: float=0.99, lam: float=0.95)->Tuple[torch.Tensor, torch.Tensor]:
    if values.dim() == 1:
        advantages = reward - values  # (batch_size,)
        returns = reward  # (batch_size,)
        return advantages.unsqueeze(-1), returns.unsqueeze(-1)
 
    batch_size, seq_len = values.shape
    rewards = torch.zeros_like(values)
    rewards[:, -1] = reward

    next_values = torch.zeros_like(values)
    next_values[:, :-1] = values[:,1:]
    deltas = rewards+gamma * next_values - values
    advantages = torch.zeros_like(values)
    gae = torch.zeros(batch_size).to(values.device)
    for t in reversed(range(seq_len)):
        gae = deltas[:, t]+gamma*lam*gae
        advantages[:, t] = gae

    returns = advantages + values 
    return advantages, returns

In [25]:
def get_log_probs(model, input_ids, attention_mask):
    """Get log probabilities for generated tokens"""
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits[:, :-1, :]
    log_probs = F.log_softmax(logits, dim=-1)
    
    next_tokens = input_ids[:, 1:]
    token_log_probs = torch.gather(log_probs, 2, next_tokens.unsqueeze(-1)).squeeze(-1)
    
    mask = attention_mask[:, 1:].float()
    token_log_probs = token_log_probs * mask
    
    return token_log_probs, mask

In [26]:
for n, p in policy_model.named_parameters():
    if 'lora' in n:
        p.requires_grad = True

In [27]:
policy_model.print_trainable_parameters()

trainable params: 786,432 || all params: 406,120,448 || trainable%: 0.1936


In [28]:
def compute_entropy(logits, mask):
    probs = F.softmax(logits, dim=-1)
    log_probs = F.log_softmax(logits, dim=-1)
    entropy = -(probs * log_probs).sum(dim=-1)
    return (entropy * mask).sum(dim=-1) / mask.sum(dim=-1)

In [29]:
def compute_reward(responses: List[str], answers: List[str]) -> torch.Tensor:
    """Better reward with partial credit"""
    from difflib import SequenceMatcher
    
    rewards = []
    for response, answer in zip(responses, answers):
        response_clean = response.lower().strip()
        answer_clean = answer.lower().strip()
        
        # Exact match in response
        if answer_clean in response_clean:
            reward = 1.0
        
        # Partial match - string similarity
        elif SequenceMatcher(None, answer_clean, response_clean).ratio() > 0.6:
            reward = 0.5
        
        # Answer words present
        elif any(word in response_clean for word in answer_clean.split() if len(word) > 2):
            reward = 0.2
        
        # At least it tried (not empty)
        elif len(response_clean) > 10:
            reward = -0.3  # Small penalty, not harsh
        
        else:
            reward = -1.0
        response_len = len(response_clean.split())
        answer_len = len(answer_clean.split())

        if response_len > 0:
            len_ratio = answer_len / response_len
            len_ratio = max(0.0, min(len_ratio, 1.5))
            reward += 0.3 * len_ratio  # weight conciseness
            
        rewards.append(reward)
    
    return torch.tensor(rewards, dtype=torch.float16)

In [30]:
def ppo_trainer(
    policy_model: PeftModel, 
    value_model: PeftModel,
    ref_model: PeftModel,
    tokenizer: PreTrainedTokenizerBase,
    dataset: Dataset,
    policy_optimizer: torch.optim.AdamW,
    value_optimizer: torch.optim.AdamW,
    reward_model: Optional[Callable[[list[str], list[str]], list[float]]] = None,
    num_epochs=3,
    rollout_batch_size=32,
    mini_batch_size=8,
    ppo_epochs=4,
    clip_eps: float=0.1,
    kl_coef: float=0.01,
    value_coef: float=0.5,
    entropy_coef: float=0.01,
    gamma: float=0.99,
    lam: float=0.95,
    max_gen_len: int=128
):
    def collate_fn(batch):
        input_ids = torch.stack([torch.tensor(item['input_ids']) for item in batch])
        attention_mask = torch.stack([torch.tensor(item['attention_mask']) for item in batch])
        answers = [item['answer'] for item in batch]
        questions = [item['question'] for item in batch]
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'answers': answers,
            'questions': questions
        }
        
    dataloader = DataLoader(dataset, batch_size=rollout_batch_size, collate_fn=collate_fn,shuffle=True)
    all_epoch_rewards = []
    

    for epoch in range(num_epochs):
        print('-----------------------------')
        print(f"Epoch {epoch+1}/{num_epochs}")
        epoch_rewards = []
        for b_idx, batch in enumerate(tqdm(dataloader, desc='Rollouts')):
            policy_model.eval()
            value_model.eval()
            ref_model.eval()
            
            rollout_buffer=[]
            with torch.no_grad():
                input_ids = batch['input_ids'].to(policy_model.device)
                attention_mask = batch['attention_mask'].to(policy_model.device)
                answers = batch['answers']
                
                response_token = policy_model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_new_tokens=max_gen_len,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.9,
                    pad_token_id=tokenizer.pad_token_id
                )
                responses=tokenizer.batch_decode(response_token, skip_special_tokens=True)
                
                rewards = compute_reward(responses, answers)
                rewards = rewards.to(policy_model.device)
                batch_avg_reward = rewards.mean().item()
                epoch_rewards.append(batch_avg_reward)

                attention_mask = (response_token != tokenizer.pad_token_id).long()

                values = value_model(
                    input_ids=response_token,
                    attention_mask=attention_mask
                ).logits.squeeze(-1)

                old_log_probs, mask = get_log_probs(
                    policy_model, response_token, attention_mask
                )
                ref_log_probs,_ = get_log_probs(
                    ref_model, response_token, attention_mask
                )
                advantages, returns = compute_gae(rewards, values, gamma, lam)
                advantages = (advantages-advantages.mean()) / (advantages.std()+1e-8)

                for i in range(len(batch['answers'])):
                    rollout_buffer.append({
                        'input_ids': response_token[i].cpu(),
                        'attention_mask': attention_mask[i].cpu(),
                        'old_log_probs': old_log_probs[i].cpu(),
                        'ref_log_probs': ref_log_probs[i].cpu(),
                        'advantages': advantages[i].cpu(),
                        'returns': returns[i].cpu(),
                        'mask': mask[i].cpu()
                    })
            policy_model.train()
            value_model.train()

            for ppo_epoch in range(ppo_epochs):
                import random
                random.shuffle(rollout_buffer)
                num_minibatches = len(rollout_buffer)//mini_batch_size

                ppo_policy_losses =[]
                ppo_value_losses =[]
                ppo_kl_penalties = []
                ppo_entropies = []
                for mb_idx in range(num_minibatches):
                    start_idx = mb_idx* mini_batch_size
                    end_idx = start_idx + mini_batch_size 
                    minibatch = rollout_buffer[start_idx:end_idx]
                    
                    #stack minibatches
                    mb_input_ids = torch.stack([item['input_ids']for item in minibatch])
                    mb_attention_mask = torch.stack([item['attention_mask'] for item in minibatch])
                    mb_old_log_probs = torch.stack([item['old_log_probs'] for item in minibatch])
                    mb_ref_log_probs = torch.stack([item['ref_log_probs'] for item in minibatch])
                    mb_advantages = torch.stack([item['advantages'] for item in minibatch])
                    mb_returns = torch.stack([item['returns'] for item in minibatch])
                    mb_mask = torch.stack([item['mask'] for item in minibatch])

                    #move to device
                    mb_input_ids = mb_input_ids.to(policy_model.device)
                    mb_attention_mask = mb_attention_mask.to(policy_model.device)
                    mb_advantages = mb_advantages.to(policy_model.device)
                    mb_old_log_probs = mb_old_log_probs.to(policy_model.device)
                    mb_ref_log_probs = mb_ref_log_probs.to(policy_model.device)
                    mb_returns = mb_returns.to(policy_model.device)
                    mb_mask = mb_mask.to(policy_model.device)
                    
                    policy_optimizer.zero_grad()
                    new_log_probs, _ = get_log_probs(
                        policy_model,
                        mb_input_ids,
                        mb_attention_mask
                    )
                    
                    #PPO loss
                    ratio = torch.exp(new_log_probs - mb_old_log_probs)    
                    surr1 = ratio*mb_advantages*mb_mask
                    surr2 = torch.clamp(ratio, 1-clip_eps, 1+clip_eps)* mb_advantages * mb_mask
                    policy_loss = -torch.min(surr1, surr2).sum() / mb_mask.sum()
                    #KL 
                    kl_div = (mb_ref_log_probs.exp() * (mb_ref_log_probs - new_log_probs)) * mb_mask
                    kl_penalty = kl_div.sum() / mb_mask.sum()
                    #entropy
                    outputs = policy_model(
                        input_ids=mb_input_ids,
                        attention_mask=mb_attention_mask
                    )
                    
                    entropy=compute_entropy(outputs.logits[:, :-1, :], mb_mask)

                    total_policy_loss = policy_loss + kl_coef * kl_penalty - entropy_coef * entropy.mean()
                    #backward
                    if not torch.isnan(total_policy_loss):
                        total_policy_loss.backward()
                        torch.nn.utils.clip_grad_norm_(policy_model.parameters(), max_norm=1.0)
                        policy_optimizer.step()

                    #value update
                    value_optimizer.zero_grad()
                    new_values = value_model(
                        input_ids=mb_input_ids,
                        attention_mask=mb_attention_mask
                    ).logits.squeeze(-1)
                    
                    # if new_values.dim() == 1:
                    #     seq_len = mb_input_ids.shape[1]
                    #     new_values = new_values.unsqueeze(-1).expand(-1, seq_len)
                    final_returns = mb_returns[:, -1]
                    #value loss
                    value_loss = value_coef * ((new_values.squeeze() - mb_returns.squeeze()) ** 2).mean()

                    # Backward and optimize
                    if not torch.isnan(value_loss):
                        value_loss.backward()
                        torch.nn.utils.clip_grad_norm_(value_model.parameters(), max_norm=1.0)
                        value_optimizer.step()

                    ppo_policy_losses.append(policy_loss.item())
                    ppo_value_losses.append(value_loss.item())
                    ppo_kl_penalties.append(kl_penalty.item())
                    ppo_entropies.append(entropy.mean().item())
                
                if b_idx % 10 == 0 and len(ppo_policy_losses) > 0:
                    print(f"\n  Batch {b_idx} - PPO Epoch {ppo_epoch+1}/{ppo_epochs}")
                    print(f"    Policy Loss: {sum(ppo_policy_losses)/len(ppo_policy_losses):.4f}")
                    print(f"    Value Loss: {sum(ppo_value_losses)/len(ppo_value_losses):.4f}")
                    print(f"    KL Div: {sum(ppo_kl_penalties)/len(ppo_kl_penalties):.4f}")
                    print(f"    Entropy: {sum(ppo_entropies)/len(ppo_entropies):.4f}")
                    print(f"    Avg Reward: {batch_avg_reward:.4f}")
            
            del rollout_buffer
            torch.cuda.empty_cache()     
            
        epoch_avg_reward = sum(epoch_rewards) / len(epoch_rewards)
        all_epoch_rewards.append(epoch_avg_reward)
        print(f"\n{'='*50}")
        print(f"Epoch {epoch+1} Average Reward: {epoch_avg_reward:.4f}")
        print(f"{'='*50}\n")
    
    return policy_model, value_model

In [31]:
policy_optimizer = torch.optim.AdamW(policy_model.parameters(), lr=1e-4)
value_optimizer = torch.optim.AdamW(value_model.parameters(), lr=1e-4)

In [32]:
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'
os.environ["TORCH_USE_CUDA_DSA"] = '1'

In [33]:
trained_policy, trained_value = ppo_trainer(
    policy_model=policy_model,
    value_model=value_model,
    ref_model=ref_model,
    tokenizer=tokenizer,
    dataset=tokenized_dataset,
    policy_optimizer=policy_optimizer,
    value_optimizer=value_optimizer,
    reward_model=None,
    num_epochs=8,  
    rollout_batch_size=16,  
    mini_batch_size=4,
    ppo_epochs=4,
    clip_eps=0.1,
    kl_coef=0.3,
    value_coef=0.5,
    entropy_coef=0.01,
    gamma=0.85,
    lam=0.87,
    max_gen_len=128 
)

-----------------------------
Epoch 1/8


Rollouts:   0%|          | 0/47 [00:00<?, ?it/s]


  Batch 0 - PPO Epoch 1/4
    Policy Loss: 0.1023
    Value Loss: 1.2231
    KL Div: 0.0027
    Entropy: 1.7889
    Avg Reward: -0.2134

  Batch 0 - PPO Epoch 2/4
    Policy Loss: 0.0887
    Value Loss: 0.5439
    KL Div: 0.0077
    Entropy: 1.8084
    Avg Reward: -0.2134

  Batch 0 - PPO Epoch 3/4
    Policy Loss: 0.0399
    Value Loss: 0.1944
    KL Div: 0.0112
    Entropy: 1.8183
    Avg Reward: -0.2134


Rollouts:   2%|▏         | 1/47 [00:24<18:53, 24.64s/it]


  Batch 0 - PPO Epoch 4/4
    Policy Loss: 0.0457
    Value Loss: 0.1589
    KL Div: 0.0097
    Entropy: 1.8161
    Avg Reward: -0.2134


...


  Batch 40 - PPO Epoch 1/4
    Policy Loss: -0.0007
    Value Loss: 0.0335
    KL Div: 0.0000
    Entropy: 0.0474
    Avg Reward: -0.1735

  Batch 40 - PPO Epoch 2/4
    Policy Loss: -0.0019
    Value Loss: 0.0354
    KL Div: 0.0000
    Entropy: 0.0475
    Avg Reward: -0.1735

  Batch 40 - PPO Epoch 3/4
    Policy Loss: -0.0015
    Value Loss: 0.0280
    KL Div: 0.0000
    Entropy: 0.0475
    Avg Reward: -0.1735


Rollouts:  87%|████████▋ | 41/47 [16:26<02:24, 24.02s/it]


  Batch 40 - PPO Epoch 4/4
    Policy Loss: -0.0031
    Value Loss: 0.0245
    KL Div: 0.0000
    Entropy: 0.0475
    Avg Reward: -0.1735


Rollouts: 100%|██████████| 47/47 [18:31<00:00, 23.65s/it]


Epoch 8 Average Reward: -0.1628



In [34]:
trained_policy.save_pretrained('/kaggle/working/best_policy/')

In [36]:
!zip -r '/kaggle/working/best_policy' '/kaggle/working/'

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/best_policy/ (stored 0%)
  adding: kaggle/working/best_policy/README.md (deflated 65%)
  adding: kaggle/working/best_policy/adapter_config.json (deflated 55%)
  adding: kaggle/working/best_policy/adapter_model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 7%)
  adding: kaggle/working/.virtual_documents/ (stored 0%)


In [37]:
test_prompt = "What is 2 + 1?"
inputs = tokenizer(test_prompt, return_tensors="pt").to('cuda')
output = trained_policy.generate(
    **inputs, 
    max_new_tokens=3,
    do_sample=False  # Greedy for testing
)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


What is 2 + 1?
-2


In [ ]:
import optuna

In [ ]:
def tune_ppo_trainer(
    policy_model: PeftModel, 
    value_model: PeftModel,
    ref_model: PeftModel,
    tokenizer: PreTrainedTokenizerBase,
    dataset: Dataset,
    policy_optimizer: torch.optim.AdamW,
    value_optimizer: torch.optim.AdamW,
    reward_model: Optional[Callable[[list[str], list[str]], list[float]]] = None,
    num_epochs=3,
    rollout_batch_size=32,
    mini_batch_size=8,
    ppo_epochs=4,
    clip_eps: float=0.1,
    kl_coef: float=0.01,
    value_coef: float=0.5,
    entropy_coef: float=0.01,
    gamma: float=0.99,
    lam: float=0.95,
    max_gen_len: int=128
):
    def collate_fn(batch):
        input_ids = torch.stack([torch.tensor(item['input_ids']) for item in batch])
        attention_mask = torch.stack([torch.tensor(item['attention_mask']) for item in batch])
        answers = [item['answer'] for item in batch]
        questions = [item['question'] for item in batch]
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'answers': answers,
            'questions': questions
        }
        
    dataloader = DataLoader(dataset, batch_size=rollout_batch_size, collate_fn=collate_fn,shuffle=True)
    all_epoch_rewards = []

    for epoch in range(num_epochs):
        print('-----------------------------')
        print(f"Epoch {epoch+1}/{num_epochs}")
        for b_idx, batch in enumerate(tqdm(dataloader, desc='Rollouts')):
            policy_model.eval()
            value_model.eval()
            ref_model.eval()
            
            rollout_buffer=[]
            epoch_rewards = []

            with torch.no_grad():
                input_ids = batch['input_ids'].to(policy_model.device)
                attention_mask = batch['attention_mask'].to(policy_model.device)
                answers = batch['answers']
                
                response_token = policy_model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_new_tokens=max_gen_len,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.9,
                    pad_token_id=tokenizer.pad_token_id
                )
                responses=tokenizer.batch_decode(response_token, skip_special_tokens=True)
                
                rewards = compute_reward(responses, answers)
                rewards = rewards.to(policy_model.device)
                batch_avg_reward = rewards.mean().item()
                epoch_rewards.append(batch_avg_reward)

                attention_mask = (response_token != tokenizer.pad_token_id).long()

                values = value_model(
                    input_ids=response_token,
                    attention_mask=attention_mask
                ).logits.squeeze(-1)

                old_log_probs, mask = get_log_probs(
                    policy_model, response_token, attention_mask
                )
                ref_log_probs,_ = get_log_probs(
                    ref_model, response_token, attention_mask
                )
                advantages, returns = compute_gae(rewards, values, gamma, lam)
                advantages = (advantages-advantages.mean()) / (advantages.std()+1e-8)

                for i in range(len(batch)):
                    rollout_buffer.append({
                        'input_ids': response_token[i].cpu(),
                        'attention_mask': attention_mask[i].cpu(),
                        'old_log_probs': old_log_probs[i].cpu(),
                        'ref_log_probs': ref_log_probs[i].cpu(),
                        'advantages': advantages[i].cpu(),
                        'returns': returns[i].cpu(),
                        'mask': mask[i].cpu()
                    })
            policy_model.train()
            value_model.train()

            for ppo_epoch in range(ppo_epochs):
                import random
                random.shuffle(rollout_buffer)
                num_minibatches = len(rollout_buffer)//mini_batch_size

                ppo_policy_losses =[]
                ppo_value_losses =[]
                ppo_kl_penalties = []
                ppo_entropies = []
                for mb_idx in range(num_minibatches):
                    start_idx = mb_idx* mini_batch_size
                    end_idx = start_idx + mini_batch_size 
                    minibatch = rollout_buffer[start_idx:end_idx]
                    
                    #stack minibatches
                    mb_input_ids = torch.stack([item['input_ids']for item in minibatch])
                    mb_attention_mask = torch.stack([item['attention_mask'] for item in minibatch])
                    mb_old_log_probs = torch.stack([item['old_log_probs'] for item in minibatch])
                    mb_ref_log_probs = torch.stack([item['ref_log_probs'] for item in minibatch])
                    mb_advantages = torch.stack([item['advantages'] for item in minibatch])
                    mb_returns = torch.stack([item['returns'] for item in minibatch])
                    mb_mask = torch.stack([item['mask'] for item in minibatch])

                    #move to device
                    mb_input_ids = mb_input_ids.to(policy_model.device)
                    mb_attention_mask = mb_attention_mask.to(policy_model.device)
                    mb_advantages = mb_advantages.to(policy_model.device)
                    mb_old_log_probs = mb_old_log_probs.to(policy_model.device)
                    mb_ref_log_probs = mb_ref_log_probs.to(policy_model.device)
                    mb_returns = mb_returns.to(policy_model.device)
                    mb_mask = mb_mask.to(policy_model.device)
                    
                    policy_optimizer.zero_grad()
                    new_log_probs, _ = get_log_probs(
                        policy_model,
                        mb_input_ids,
                        mb_attention_mask
                    )
                    
                    #PPO loss
                    ratio = torch.exp(new_log_probs - mb_old_log_probs)    
                    surr1 = ratio*mb_advantages*mb_mask
                    surr2 = torch.clamp(ratio, 1-clip_eps, 1+clip_eps)* mb_advantages * mb_mask
                    policy_loss = -torch.min(surr1, surr2).sum() / mb_mask.sum()
                    #KL 
                    kl_div = (new_log_probs - mb_ref_log_probs)*mb_mask
                    kl_penalty = kl_div.sum() / mb_mask.sum()
                    #entropy
                    outputs = policy_model(
                        input_ids=mb_input_ids,
                        attention_mask=mb_attention_mask
                    )
                    
                    entropy=compute_entropy(outputs.logits[:, :-1, :], mb_mask)

                    total_policy_loss = policy_loss + kl_coef * kl_penalty - entropy_coef * entropy.mean()
                    #backward
                    if not torch.isnan(total_policy_loss):
                        total_policy_loss.backward()
                        torch.nn.utils.clip_grad_norm_(policy_model.parameters(), max_norm=1.0)
                        policy_optimizer.step()

                    #value update
                    value_optimizer.zero_grad()
                    new_values = value_model(
                        input_ids=mb_input_ids,
                        attention_mask=mb_attention_mask
                    ).logits.squeeze(-1)
                    
                    # if new_values.dim() == 1:
                    #     seq_len = mb_input_ids.shape[1]
                    #     new_values = new_values.unsqueeze(-1).expand(-1, seq_len)
                    final_returns = mb_returns[:, -1]
                    #value loss
                    # value_loss = ((new_values-final_returns)**2*mb_mask).sum()/ mb_mask.sum()
                    value_loss = value_coef * ((new_values - final_returns) ** 2).mean()


                    # Backward and optimize
                    if not torch.isnan(value_loss):
                        value_loss.backward()
                        torch.nn.utils.clip_grad_norm_(value_model.parameters(), max_norm=1.0)
                        value_optimizer.step()

                    ppo_policy_losses.append(policy_loss.item())
                    ppo_value_losses.append(value_loss.item())
                    ppo_kl_penalties.append(kl_penalty.item())
                    ppo_entropies.append(entropy.mean().item())
                
                if b_idx % 10 == 0:
                    print(f"\n  Batch {b_idx} - PPO Epoch {ppo_epoch+1}/{ppo_epochs}")
                    print(f"    Policy Loss: {sum(ppo_policy_losses)/len(ppo_policy_losses):.4f}")
                    print(f"    Value Loss: {sum(ppo_value_losses)/len(ppo_value_losses):.4f}")
                    print(f"    KL Div: {sum(ppo_kl_penalties)/len(ppo_kl_penalties):.4f}")
                    print(f"    Entropy: {sum(ppo_entropies)/len(ppo_entropies):.4f}")
                    print(f"    Avg Reward: {batch_avg_reward:.4f}")
            
            del rollout_buffer
            torch.cuda.empty_cache()     
            
        epoch_avg_reward = sum(epoch_rewards) / len(epoch_rewards)
        all_epoch_rewards.append(epoch_avg_reward)
        print(f"\n{'='*50}")
        print(f"Epoch {epoch+1} Average Reward: {epoch_avg_reward:.4f}")
        print(f"{'='*50}\n")
    
    final_avg_reward = sum(all_epoch_rewards) / len(all_epoch_rewards)
    print(f"\n{'='*50}")
    print(f"FINAL AVERAGE REWARD: {final_avg_reward:.4f}")
    print(f"{'='*50}\n")

    return final_avg_reward

In [ ]:
def objective(trial):

    clip_eps = trial.suggest_categorical('clip_eps', [0.1, 0.15, 0.2, 0.3])
    kl_coef = trial.suggest_categorical('kl_coef', [0.1, 0.2, 0.3, 0.4])
    value_coef = trial.suggest_categorical('value_coef', [0.5, 0.4, 0.6, 0.3])
    entropy_coef = trial.suggest_categorical('entropy_coef', [0.01, 0.03, 0.05, 0.04])
    gamma = trial.suggest_categorical('gamma', [0.99, 0.90, 0.85, 0.70])
    lam = trial.suggest_categorical('lam', [0.95, 0.87, 0.80, 0.76])
    # rollout_batch_size = trial.suggest_categorical('rollout_batch_size'. [8, 16, 32])
    # mini_batch_size = 
    policy_lr = trial.suggest_loguniform('policy_lr', 5e-5, 1e-3)
    value_lr = trial.suggest_loguniform('value_lr', 5e-5, 1e-3)
    
    print(f"\n{'='*60}")
    print(f"TRIAL {trial.number}")
    print(f"{'='*60}")
    print(f"Hyperparameters:")
    print(f"  clip_eps: {clip_eps}")
    print(f"  kl_coef: {kl_coef}")
    print(f"  value_coef: {value_coef}")
    print(f"  entropy_coef: {entropy_coef}")
    print(f"  gamma: {gamma}")
    print(f"  lam: {lam}")
    print(f"  policy_lr: {policy_lr:.2e}")
    print(f"  value_lr: {value_lr:.2e}")
    print(f"{'='*60}\n")

    trial_policy_model = PeftModel.from_pretrained(
        base_model,
        "/kaggle/input/loraadapters/pytorch/default/1",
        trainable=True
    )
    
    trial_value_model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=1,
        torch_dtype=torch.float16
    )
    for n,p in trial_policy_model.named_parameters():
        if 'lora' in n:
            p.requires_grad=True
            
    trial_value_model.config.pad_token_id = tokenizer.pad_token_id
    trial_value_model = get_peft_model(trial_value_model, value_config)
    trial_value_model.to('cuda')
    
    policy_optimizer = torch.optim.AdamW(trial_policy_model.parameters(), lr=policy_lr)
    value_optimizer = torch.optim.AdamW(trial_value_model.parameters(), lr=value_lr)
    
    small_dataset = tokenized_dataset.select(range(min(200, len(tokenized_dataset))))
    try:
        average_reward = tune_ppo_trainer(
            policy_model=trial_policy_model,
            value_model=trial_value_model,
            ref_model=ref_model,
            tokenizer=tokenizer,
            dataset=small_dataset,
            policy_optimizer=policy_optimizer,
            value_optimizer=value_optimizer,
            reward_model=None,
            num_epochs=1,  # Just 1 epoch for tuning
            rollout_batch_size=8,  # Smaller for faster tuning
            mini_batch_size=4,
            ppo_epochs=4,
            clip_eps=clip_eps,
            kl_coef=kl_coef,
            value_coef=value_coef,
            entropy_coef=entropy_coef,
            gamma=gamma,
            lam=lam,
            max_gen_len=64  # Shorter generation for speed
        )
    except Exception as e:
        print(f"Trial {trial.number} failed with error: {e}")
        return -10.0
    finally:
        del trial_policy_model, trial_value_model
        torch.cuda.empty_cache()
    return average_reward

In [ ]:
import optuna

In [ ]:
print("Starting Optuna hyperparameter search...")
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20, timeout=3600*6)

In [ ]:
best_params = study.best_params

In [ ]:
policy_optimizer = torch.optim.AdamW(policy_model.parameters(), lr=best_params['policy_lr'])
value_optimizer = torch.optim.AdamW(value_model.parameters(), lr=best_params['value_lr'])

In [ ]:
best_params

In [ ]:
trained_policy, trained_value = ppo_trainer(
    policy_model=trial_policy_model,
    value_model=trial_value_model,
    ref_model=ref_model,
    tokenizer=tokenizer,
    dataset=small_dataset,
    policy_optimizer=policy_optimizer,
    value_optimizer=value_optimizer,
    reward_model=None,
    num_epochs=8,  
    rollout_batch_size=32,  
    mini_batch_size=8,
    ppo_epochs=4,
    clip_eps=best_params['clip_eps'],
    kl_coef=best_params['kl_coef'],
    value_coef=best_params['value_coef'],
    entropy_coef=best_params['entropy_coef'],
    gamma=best_params['gamma'],
    lam=best_params['lam'],
    max_gen_len=128 
)
